This is a notebook intended to contain explanations on numpy things.

# Numpy Broadcasting

Numpy broadcasting is simply a mechanism to allow array expansion without duplication. It's extremely useful for performing operations on arrays of different sizes and automatically executing operations in parallel.

It's useful to think of Numpy Arrays just as containers for data. Just like a database or something else there's more than 1 way to set it up and encode the information. This freedom allows for more solutions but requires documentation for which permutation is being used. 

<h3>An example of solving for distance between all points in an array:</h3>

What we want here is to take an array, X, filled with points in 2-d and get the distance from each point to every other point. While we could use for-loops numpy lets us do this in parallel quickly. So we know numpy has the tools, unfortunately school tells us how to solve certain problems, not which problems apply to which realities. That's where practice and tutorials like this come in handy. So lets formalize the problem into something Numpy can solve.

We want $X[i]-X[j]$ for all i,j in range(0,number of rows). This gives us the distance between every point in an array. But there are 2 problems we have to solve: how do we store this information, and how do we perform this operation in parallel? Well we know Numpy can do component-wise operations in parallel, so the question is how do we make the components 'line up' so we can simply execute A-B for some A and some B? This is where broadcasting comes to the rescue!

Let's take this step by step. $X[i]-X[j]$ in Numpy is implicitly $X[i][k]-X[j][k]$. So let's try and run this naively. 

In [16]:
import numpy as np
np.random.seed(3)
X = np.random.randint(1,10,size = (10,2))

X[:]-X[:]

array([[0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0]])

So what happened here? Well clearly we didn't index over $i$ and $j$ separately. By using ':' inside the index for the first access we told numpy to access every row in order, and then subtract from that every row <i>also in order</i>. Clearly we need a way to make $i$ line up with every $j$.

But this would generate a new array for every combination of i, j, and k. Since we have 3 indicies we know our storage object needs to similarly be accessed by 3 indicies, lest we leave some results inaccessible. For now we ignore the symmetry of the problem. So lets choose to store it in a 3d array called $D$. Keep in mind this is a <i>choice</i> we could have stored it in some python object or some dataframe equally well. 

So given we've chosen a 3d array and an operation $X[i][k]-X[j][k]$ we must map this into $D[i][j][k]$. One observation we can make is that j does not affect our first term and i does not affect our second term. So for the first term, this means if we 'duplicate' X along a new axis we can get $X[i][j][k] = X[i][k]$ for all i,j,k. Similarly this also allows us to get $X[i][j][k] = X[j][k]$ for all i,j,k. 

In [5]:
 # generate ten 2-dim vectors at random
#Generate distances between each point.
#What we want: X[k]-X[i] for all k, i. More explicitly X[k][j]-X[i][j] where j is 0(x coord) or 1(y coord).
#Need to choose a way to store this information. 3 indicies written = 3d array needed. 
#Shape of 3d: some combination of 10, 10 and 2 because thats the range of the indicies.

"""Natural approach
Storage choice:Y[0][i][j]=distance from point 0 to point i along dimension j. Shape = 1x10x2
In general: Y[k][i][j]=distance from point k to point i along dimension j. Shape = 10x10x2
How to get: Y[k][i][j] must equal X[k][j]-X[i][j]. Make indicies match: X[k][j] -> X[k][0][j], X[i][j] -> X[0][i][j])
How to get indicies to match: use np.newaxis where you want a 0. 
Note that rows/columns are not switched. Can put colons in order without problem. 
Arrays will broadcase along 0ed axes if needed to execute operation in parallel 
"""

Y=X[:,np.newaxis,:] - X[np.newaxis, :,:] #Cleanest solution

"""Alt Approach 1: just switch indicies around!
Storage choice: Z[i][j][0]=distance from point 0 to point i along dimension j. Shape = 10x2x1
In general: Z[i][j][k]=distance from point k to point i along dimension j. Shape = 10x2x10
How to get: Z[i][j][k] must equal X[k][j]-X[i][j]. Make indicies match: X[k][j] -> X[0][j][k], X[i][j] -> X[i][j][0]
How to get indicies to match: use np.newaxis where you want a 0. 
Note that rows/columns switched in first term. Must use a transpose on first term. 
Arrays will broadcase along 0ed axes if needed to execute in parallel.
"""

Z=X.T[np.newaxis,:,:] - X[:,:,np.newaxis] #transpose taken b/c row(X.t)=col(X) and first needs Xs cols first
Z_t=np.transpose(Z, axes=[2,0,1]) #Transpose to match Y. [i][j][k]-> [k][i][j]

"""Alt Approach 2: just switch indicies around again!
Storage choice: W[j][0][i]=distance from point 0 to point i along dimension j. Shape = 2x1x10
In general: W[j][k][i]=distance from point k to point i along dimension j. Shape = 2x10x10
How to get: W[j][k][i] must equal X[k][j]-X[i][j]. Make indicies match: X[k][j] -> X[j][k][0], X[i][j] -> X[j][0][i]
How to get indicies to match: use np.newaxis where you want a 0.
Note that rows/columns switched in both terms. Must use a transpose on both terms.
Arrays will broadcase along 0ed axes if needed to execute in parallel.
"""

W=X.T[:,:,np.newaxis] - X.T[:,np.newaxis,:] #transpose taken b/c row(X.t)=col(X) and both needs Xs cols first
W_t=np.transpose(W, axes=[1,2,0]) #Transpose to match Y. [j][k][i] -> [k][i][j]



enter desired slice (0-9):1
Slice: 1
[[0 0 8 5 1 8 1 7 6 7]
 [5 0 3 3 2 4 0 2 6 5]]
[[0 0 8 5 1 8 1 7 6 7]
 [5 0 3 3 2 4 0 2 6 5]]
[[0 0 8 5 1 8 1 7 6 7]
 [5 0 3 3 2 4 0 2 6 5]]


In [14]:
#verify that all the slices are identical
i = int(input("Enter a slice to print (0-9):"))
print("Slice: "+str(i))
print(Y[i].T)
print(Z_t[i].T)
print(W_t[i].T)
#check if both Y==Z_t and Y==W_t
truthval=np.array_equal(Y,Z_t)&np.array_equal(Y,W_t)
print(f"In general all slices are the same is a {truthval} statement")

Enter a slice to print (0-9):5
Slice: 5
[[-8 -8  0 -3 -7  0 -7 -1 -2 -1]
 [ 1 -4 -1 -1 -2  0 -4 -2  2  1]]
[[-8 -8  0 -3 -7  0 -7 -1 -2 -1]
 [ 1 -4 -1 -1 -2  0 -4 -2  2  1]]
[[-8 -8  0 -3 -7  0 -7 -1 -2 -1]
 [ 1 -4 -1 -1 -2  0 -4 -2  2  1]]
In general all slices are the same is a False statement
